# Ensemble Sentiment Model Test

This notebook demonstrates how to load and use the ensemble sentiment model that combines LSTM feature extraction with traditional classifiers.

In [ ]:
import json
import pickle
import numpy as np
import os

# Check if models directory exists
print("Checking models directory...")
ensemble_model_path = "models/ensemble_sentiment_model"
if os.path.exists(ensemble_model_path):
    print(f"Found ensemble model path: {ensemble_model_path}")
    print("Contents:")
    print(os.listdir(ensemble_model_path))
else:
    print(f"Ensemble model path not found: {ensemble_model_path}")

In [ ]:
# Try to load TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    print("TensorFlow version:", tf.__version__)
    print("TensorFlow/Keras imports successful")
except ImportError:
    try:
        import keras
        from keras.models import load_model
        from keras.preprocessing.sequence import pad_sequences
        print("Keras version:", keras.__version__)
        print("Keras imports successful")
    except ImportError:
        print("Failed to import Keras or TensorFlow. Will use fallback method.")

In [ ]:
# Load model components
def load_ensemble_model():
    try:
        # Load LSTM feature extractor
        lstm_feature_extractor = load_model('models/ensemble_sentiment_model/lstm_feature_extractor.h5')
        print("LSTM feature extractor loaded")
        
        # Try to load full ensemble model
        try:
            ensemble_model = load_model('models/ensemble_sentiment_model/full_ensemble_model.h5')
            print("Full ensemble model loaded")
        except Exception as e:
            print(f"Could not load full ensemble model: {str(e)}")
            ensemble_model = None
        
        # Load classifiers if full ensemble model isn't available
        classifiers = {}
        if ensemble_model is None:
            try:
                import joblib
                # Load the individual classifiers
                classifiers["random_forest"] = joblib.load('models/ensemble_sentiment_model/best_classifier_random_forest.pkl')
                print("Random Forest classifier loaded")
                
                classifiers["gradient_boosting"] = joblib.load('models/ensemble_sentiment_model/best_classifier_gradient_boosting.pkl')
                print("Gradient Boosting classifier loaded")
                
                classifiers["logistic_regression"] = joblib.load('models/ensemble_sentiment_model/best_classifier_lojistik_regresyon.pkl')
                print("Logistic Regression classifier loaded")
            except Exception as e:
                print(f"Error loading classifier models: {str(e)}")
                return None, None, None, None
        
        # Load tokenizer and config
        with open('models/ensemble_sentiment_model/tokenizer.pkl', 'rb') as f:
            tokenizer = pickle.load(f)
            print("Tokenizer loaded")
        
        with open('models/ensemble_sentiment_model/config.json', 'r') as f:
            config = json.load(f)
            print("Config loaded:", config)
        
        return lstm_feature_extractor, ensemble_model, tokenizer, config, classifiers
    except Exception as e:
        print(f"Error loading model components: {str(e)}")
        return None, None, None, None, {}

# Load the models
lstm_feature_extractor, ensemble_model, tokenizer, config, classifiers = load_ensemble_model()

In [ ]:
# Import text preprocessing from sentiment_model
import sys
import os
sys.path.append(os.path.abspath("sentiment_model"))
from text_preprocessing import preprocess_text

# Test samples
test_samples = [
    "This product is amazing! I love it.",
    "I hate this, it's terrible.",
    "Average product, could be better.",
    "The service was fast and helpful.",
    "Completely useless, don't waste your money."
]

# Process and predict
for text in test_samples:
    print(f"Original: {text}")
    processed_text = preprocess_text(text)
    print(f"Processed: {processed_text}")
    
    # Tokenize and pad
    sequences = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(
        sequences, 
        maxlen=config.get('max_sequence_length', 50),
        padding='post'
    )
    
    # Extract LSTM features
    lstm_features = lstm_feature_extractor.predict(padded_sequence)
    
    # Make prediction using either the full ensemble model or the best classifier
    if ensemble_model is not None:
        # Full ensemble model is available
        prediction = ensemble_model.predict(padded_sequence)[0][0]
        model_used = "full_ensemble_model"
    else:
        # Use the best classifier from config
        best_classifier_name = config.get('best_classifier', 'Random Forest').lower().replace(' ', '_')
        model_used = best_classifier_name
        
        if best_classifier_name == 'random_forest' and 'random_forest' in classifiers:
            prediction = classifiers['random_forest'].predict_proba(lstm_features)[0][1]
        elif best_classifier_name == 'gradient_boosting' and 'gradient_boosting' in classifiers:
            prediction = classifiers['gradient_boosting'].predict_proba(lstm_features)[0][1]
        elif best_classifier_name == 'logistic_regression' and 'logistic_regression' in classifiers:
            prediction = classifiers['logistic_regression'].predict_proba(lstm_features)[0][1]
        else:
            # If best classifier not found, use any available classifier
            for classifier_name, classifier in classifiers.items():
                prediction = classifier.predict_proba(lstm_features)[0][1]
                model_used = classifier_name
                break
    
    # Convert prediction to sentiment and confidence
    if prediction >= 0.5:
        sentiment = "Pozitif"
        confidence = float(prediction)
    else:
        sentiment = "Negatif"
        confidence = float(1.0 - prediction)
    
    print(f"Sentiment: {sentiment}, Confidence: {confidence*100:.2f}%, Model used: {model_used}")
    print("-" * 50)

## Testing the API

Below we'll test the API endpoints to make sure that the model is properly integrated into the Flask application.

In [ ]:
import requests

# Start the Flask app in a separate process before running this
# The app should be running on http://127.0.0.1:5000

def test_sentiment_api():
    url = "http://127.0.0.1:5000/analyze"
    
    # Sample texts to analyze
    texts = [
        "This product is amazing! I love it.",
        "I hate this, it's terrible.",
        "Average product, could be better.",
        "The service was fast and helpful.",
        "Completely useless, don't waste your money."
    ]
    
    print("Testing the API endpoint...")
    
    for text in texts:
        data = {"text": text}
        try:
            response = requests.post(url, json=data)
            if response.status_code == 200:
                result = response.json()
                print(f"Original: {text}")
                print(f"Sentiment: {result['sentiment']}, Confidence: {result['confidence']}%")
                print("-" * 50)
            else:
                print(f"Error: {response.status_code}, {response.text}")
        except Exception as e:
            print(f"API request failed: {str(e)}")
            
    print("API testing complete.")

# Uncomment below to test the API (requires Flask app to be running)
# test_sentiment_api()